In [2]:
rdd = sc.textFile("../data/taxi-data-sorted-small.csv.bz2")

In [3]:
rdd.take(5)

['07290D3599E7A0D62097A346EFCC1FB5,E7750A37CAB07D0DFF0AF7E3573AC141,2013-01-01 00:00:00,2013-01-01 00:02:00,120,0.44,-73.956528,40.716976,-73.962440,40.715008,CSH,3.50,0.50,0.50,0.00,0.00,4.50',
 '22D70BF00EEB0ADC83BA8177BB861991,3FF2709163DE7036FCAA4E5A3324E4BF,2013-01-01 00:02:00,2013-01-01 00:02:00,0,0.00,0.000000,0.000000,0.000000,0.000000,CSH,27.00,0.00,0.50,0.00,0.00,27.50',
 '0EC22AAF491A8BD91F279350C2B010FD,778C92B26AE78A9EBDF96B49C67E4007,2013-01-01 00:01:00,2013-01-01 00:03:00,120,0.71,-73.973145,40.752827,-73.965897,40.760445,CSH,4.00,0.50,0.50,0.00,0.00,5.00',
 '1390FB380189DF6BBFDA4DC847CAD14F,BE317B986700F63C43438482792C8654,2013-01-01 00:01:00,2013-01-01 00:03:00,120,0.48,-74.004173,40.720947,-74.003838,40.726189,CSH,4.00,0.50,0.50,0.00,0.00,5.00',
 '3B4129883A1D05BE89F2C929DE136281,7077F9FD5AD649AEACA4746B2537E3FA,2013-01-01 00:01:00,2013-01-01 00:03:00,120,0.61,-73.987373,40.724861,-73.983772,40.730995,CRD,4.00,0.50,0.50,0.00,0.00,5.00']

In [4]:
rdd = rdd.map(lambda x: x.split(','))

In [5]:
def isfloat(value):
    try:
        float(value)
        return True
    except:
        return False

def correctRows(p):
    if(len(p) == 17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[5]) != 0 and float(p[11]) != 0):
                return p

In [6]:
texilinesCorrect = rdd.filter(correctRows)

In [8]:
texilinesCorrect.take(1)

[['07290D3599E7A0D62097A346EFCC1FB5',
  'E7750A37CAB07D0DFF0AF7E3573AC141',
  '2013-01-01 00:00:00',
  '2013-01-01 00:02:00',
  '120',
  '0.44',
  '-73.956528',
  '40.716976',
  '-73.962440',
  '40.715008',
  'CSH',
  '3.50',
  '0.50',
  '0.50',
  '0.00',
  '0.00',
  '4.50']]

In [62]:
n = texilinesCorrect.count()

In [63]:
n

1990270

In [89]:
def getProductandPower(pair):
    productPart = pair[1][0]*pair[1][1]
    powerPart =pow(pair[1][0], 2)
    return (pair[0], (productPart, powerPart))

In [90]:
distance_money = texilinesCorrect.map(lambda x: (1, (float(x[5]), float(x[11]))))

In [91]:
distance_money.take(1)

[(1, (0.44, 3.5))]

In [92]:
productPowerPart = distance_money.map(getProductandPower)\
                                .reduceByKey(lambda x1, x2: (x1[0]+x2[0], x1[1]+x2[1]))

In [93]:
productPower = productPowerPart.collect()

In [94]:
productPower

[(1, (131824919.4254002, 40345270.133203365))]

In [95]:
sumXandYPart = distance_money.reduceByKey(lambda x1, x2: (x1[0]+x2[0], x1[1]+x2[1]))

In [99]:
sumXandY = sumXandYPart.collect(1)

In [100]:
sumXandY

[(1, (5779977.3200001195, 23499862.04))]

In [101]:
m = (n * productPower[0][1][0] - sumXandY[0][1][0] * sumXandY[0][1][1]) /\
    (n * productPower[0][1][1] - sumXandY[0][1][0]**2)

In [102]:
m

2.698633750085706

In [105]:
b = (productPower[0][1][1] * sumXandY[0][1][1] - sumXandY[0][1][0] *  productPower[0][1][0]) /\
    (n * productPower[0][1][1] - sumXandY[0][1][0]**2)

In [106]:
b

3.9702252305052834